# Análise exploratória e teste amostral - linha 844

**TODO:**
- (1) Tabela pré-reprocessamento
- (2) Gráficos da análise exploratória
- (3) Tabela pós-reprocessamento

Preparar o ambiente:

In [1]:
#!pip install matplotlib
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import timedelta, datetime

pd.set_option('display.max_rows', 25)

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve().parent

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'
paths["output"] = current_path / 'data' / 'output'
paths["figures"] = current_path / 'data' / 'figures'
paths["scripts"] = current_path / 'scripts'


for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)      

if paths["scripts"] not in sys.path:
    sys.path.append(str(paths["scripts"]))
 
from graphs import *
from categorize_trips import *

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"


# (1) Tabela pré-reprocessamento

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pre_solucao.xlsx`

> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao	datetime_chegada_apuracao	status

In [2]:
servico = "844"

#### 1 - Importar e tratar os dados da amostra

In [3]:
amostra = pd.read_excel('../data/raw/Relatorio de Viagens das linhas 605_LECD50. 663 e 844.xlsx')

# padronizar as colunas da amostra 
amostra = amostra.rename(
    columns={"Data": "data",
             "Hora_Inicio": "hora_inicio",
             "Linha": "servico",
             "Veiculo": "id_veiculo",
             "Hora_Fim": "hora_fim",
             "Sentido": "sentido"
             }
    )

# setar tipos de dados e criar colunas com o datetime
amostra['servico'] = amostra['servico'].astype(str)
amostra['data'] = amostra['data'].astype(str)
amostra['hora_inicio'] = amostra['hora_inicio'].astype(str)
amostra['datetime_partida'] = pd.to_datetime(amostra['data'] + ' ' + amostra['hora_inicio'])
amostra['hora_fim'] = amostra['hora_fim'].astype(str)
amostra['datetime_chegada'] = pd.to_datetime(amostra['data'] + ' ' + amostra['hora_fim'])
amostra['id_veiculo'] = amostra['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)
amostra_tratada =  amostra[amostra['servico'] == servico]

amostra_tratada

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada
128,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00
129,2022-09-15,844,47689,10:42:00,10:50:00,V,2022-09-15 10:42:00,2022-09-15 10:50:00
130,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00
131,2022-09-15,844,47689,11:03:00,11:10:00,V,2022-09-15 11:03:00,2022-09-15 11:10:00
132,2022-09-15,844,47689,11:10:00,11:21:00,I,2022-09-15 11:10:00,2022-09-15 11:21:00
...,...,...,...,...,...,...,...,...
403,2023-07-19,844,47689,17:48:00,17:59:00,V,2023-07-19 17:48:00,2023-07-19 17:59:00
404,2023-07-19,844,47689,18:01:00,18:16:00,I,2023-07-19 18:01:00,2023-07-19 18:16:00
405,2023-07-19,844,47689,18:18:00,18:33:00,V,2023-07-19 18:18:00,2023-07-19 18:33:00
406,2023-07-19,844,47689,18:35:00,18:48:00,I,2023-07-19 18:35:00,2023-07-19 18:48:00


In [4]:
amostra_tratada_2022 = amostra_tratada[amostra_tratada['data'] <= '2022-12-31']
amostra_tratada_2023 = amostra_tratada[amostra_tratada['data'] > '2022-12-31']
datas = amostra_tratada['data'].unique()
veiculos = amostra_tratada['id_veiculo'].unique()

print("A quantidade total de viagens na amostra é:", amostra_tratada.shape[0])
print("A quantidade de viagens na amostra em 2022 é:", amostra_tratada_2022.shape[0])
print("A quantidade de viagens na amostra em 2023 é:", amostra_tratada_2023.shape[0])
print("A amostra é referente aos dias:", datas)
print("A amostra tem dados dos seguintes veículos:", veiculos) 

A quantidade total de viagens na amostra é: 280
A quantidade de viagens na amostra em 2022 é: 136
A quantidade de viagens na amostra em 2023 é: 144
A amostra é referente aos dias: ['2022-09-15' '2022-09-20' '2022-09-28' '2023-07-06' '2023-07-11'
 '2023-07-19']
A amostra tem dados dos seguintes veículos: ['47689']


#### 2 - Remover viagens sobrepostas da amostra

Esta etapa classifica como "Viagem inválida - sobreposição de viagem" aquelas viagens em que um mesmo `id_veiculo` realiza múltiplas viagens em um mesmo intervalo de tempo (`datetime_partida` e `datetime_chegada`).

No caso de duas ou mais viagens sobrepostas, apenas uma será classificada nas demais etapas de classificação do notebook.

In [5]:
amostra_deduplicada = remove_overlapping_trips(amostra_tratada)
amostra_deduplicada

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada,status
128,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00,NaN
129,2022-09-15,844,47689,10:42:00,10:50:00,V,2022-09-15 10:42:00,2022-09-15 10:50:00,NaN
130,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00,NaN
131,2022-09-15,844,47689,11:03:00,11:10:00,V,2022-09-15 11:03:00,2022-09-15 11:10:00,NaN
132,2022-09-15,844,47689,11:10:00,11:21:00,I,2022-09-15 11:10:00,2022-09-15 11:21:00,NaN
...,...,...,...,...,...,...,...,...,...
403,2023-07-19,844,47689,17:48:00,17:59:00,V,2023-07-19 17:48:00,2023-07-19 17:59:00,NaN
404,2023-07-19,844,47689,18:01:00,18:16:00,I,2023-07-19 18:01:00,2023-07-19 18:16:00,NaN
405,2023-07-19,844,47689,18:18:00,18:33:00,V,2023-07-19 18:18:00,2023-07-19 18:33:00,NaN
406,2023-07-19,844,47689,18:35:00,18:48:00,I,2023-07-19 18:35:00,2023-07-19 18:48:00,NaN


In [6]:
count_non_nan = amostra_deduplicada['status'].notna().sum()
count_non_nan

0

### 3 Comparar dados da amostra com os dados das viagens completas

#### 3.1 Consultar se as viagens da amostra foram feitas pelos veículos indicados na amostra

In [7]:
# separar as datas da amostra para fazer a query
datas = amostra_deduplicada['data'].unique()

q = f"""
       SELECT
         id_veiculo, servico_informado, sentido, datetime_partida, datetime_chegada
       FROM
         `rj-smtr.projeto_subsidio_sppo.viagem_completa`
       WHERE
         data IN {tuple(datas)}
         AND servico_informado = '{servico}'
       """
       
viagem_completa_prod = bd.read_sql(q, from_file=True)
viagem_completa_prod.info() 

viagem_completa_prod['id_veiculo'] = viagem_completa_prod['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)
viagem_completa_prod      

Downloading: 100%|██████████| 95/95 [00:00<00:00, 268.36rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id_veiculo         95 non-null     object        
 1   servico_informado  95 non-null     object        
 2   sentido            95 non-null     object        
 3   datetime_partida   95 non-null     datetime64[ns]
 4   datetime_chegada   95 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(3)
memory usage: 3.8+ KB


,id_veiculo,servico_informado,sentido,datetime_partida,datetime_chegada
0,47689,844,C,2022-09-20 12:17:24,2022-09-20 12:28:54
1,47689,844,C,2022-09-20 15:38:24,2022-09-20 15:50:54
2,47689,844,C,2022-09-20 18:14:24,2022-09-20 18:27:24
3,47689,844,C,2022-09-20 11:14:24,2022-09-20 11:25:54
4,47689,844,C,2022-09-20 12:56:54,2022-09-20 13:09:24
...,...,...,...,...,...
90,47689,844,C,2023-07-19 17:48:39,2023-07-19 18:11:09
91,47689,844,C,2023-07-19 11:56:39,2023-07-19 12:09:09
92,47689,844,C,2023-07-19 13:32:09,2023-07-19 13:42:39
93,47689,844,C,2023-07-19 17:23:09,2023-07-19 17:42:09


In [8]:
datas

array(['2022-09-15', '2022-09-20', '2022-09-28', '2023-07-06',
       '2023-07-11', '2023-07-19'], dtype=object)

Esta etapa cruza dados da amostra com as viagens completas para o serviço 844 de acordo com os seguintes critérios:

- o datetime_partida da viagem que consta na tabela de viagens completas deve estar dentro 
do intervalo de mais ou menos 4 minutos do datetime_partida que consta no amostra.

In [9]:
viagem_completa_prod['datetime_partida'] = pd.to_datetime(viagem_completa_prod['datetime_partida'])

amostra_classificada = check_trips(amostra_deduplicada, viagem_completa_prod, 4,
                                   "Viagem circular identificada e já paga")
amostra_classificada


Casos duplicados encontrados no cruzamento de dados:
         data servico_amostra id_veiculo_amostra hora_inicio  hora_fim  \
0  2022-09-28             844              47689    10:46:00  10:52:00   
1  2022-09-28             844              47689    10:52:00  11:00:00   

  sentido_amostra datetime_partida_amostra datetime_chegada_amostra  \
0               I      2022-09-28 10:46:00      2022-09-28 10:52:00   
1               V      2022-09-28 10:52:00      2022-09-28 11:00:00   

                                   status id_veiculo_apurado servico_apurado  \
0  Viagem circular identificada e já paga              47689             844   
1  Viagem circular identificada e já paga              47689             844   

  sentido_apurado datetime_partida_apurado datetime_chegada_apurado  
0               C      2022-09-28 10:48:24      2022-09-28 10:57:24  
1               C      2022-09-28 10:48:24      2022-09-28 10:57:24  


,data,servico_amostra,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
0,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00,NaN,NaN,NaN,NaN,NaT,NaT
1,2022-09-15,844,47689,10:42:00,10:50:00,V,2022-09-15 10:42:00,2022-09-15 10:50:00,NaN,NaN,NaN,NaN,NaT,NaT
2,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00,NaN,NaN,NaN,NaN,NaT,NaT
3,2022-09-15,844,47689,11:03:00,11:10:00,V,2022-09-15 11:03:00,2022-09-15 11:10:00,NaN,NaN,NaN,NaN,NaT,NaT
4,2022-09-15,844,47689,11:10:00,11:21:00,I,2022-09-15 11:10:00,2022-09-15 11:21:00,NaN,NaN,NaN,NaN,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-07-19,844,47689,16:26:00,16:37:00,V,2023-07-19 16:26:00,2023-07-19 16:37:00,Viagem circular identificada e já paga,47689,844,C,2023-07-19 16:28:09,2023-07-19 16:48:39
276,2023-07-19,844,47689,17:01:00,17:10:00,V,2023-07-19 17:01:00,2023-07-19 17:10:00,Viagem circular identificada e já paga,47689,844,C,2023-07-19 17:02:39,2023-07-19 17:15:09
277,2023-07-19,844,47689,17:21:00,17:37:00,V,2023-07-19 17:21:00,2023-07-19 17:37:00,Viagem circular identificada e já paga,47689,844,C,2023-07-19 17:23:09,2023-07-19 17:42:09
278,2023-07-19,844,47689,17:48:00,17:59:00,V,2023-07-19 17:48:00,2023-07-19 17:59:00,Viagem circular identificada e já paga,47689,844,C,2023-07-19 17:48:39,2023-07-19 18:11:09


In [10]:
count_non_nan = amostra_classificada['status'].notna().sum()
count_non_nan

73

In [11]:
# Corrigir linha duplicada
condition = (
    (amostra_classificada['id_veiculo_amostra'] == '47689') & 
    (amostra_classificada['datetime_partida_amostra'] == "2022-09-28 10:52:00")
)

# Update the columns from 'status' till the end for those rows
amostra_classificada.loc[condition, 'status':] = np.nan

Classificar as viagens da amostra de ida que na verdade foram identificadas como circulares.

Para o ano de 2023, apenas as viagens no sentido volta foram identificadas. Ao verificar a duração destas viagens, nota-se que elas incluem as viagens classificadas como ida, ou seja, a amostra considera como duas viagens separadas o que para a gente é apenas uma viagem circular.

Identificar as metades das viagens circulares:

In [12]:
# Ordenando o DataFrame pela coluna 'datetime_partida'
amostra_classificada.sort_values('datetime_partida_amostra', inplace=True)

# Resetando o índice para facilitar a iteração
amostra_classificada.reset_index(drop=True, inplace=True)

# Certifique-se de que a coluna datetime_partida_amostra é do tipo datetime
amostra_classificada['datetime_partida_amostra'] = pd.to_datetime(amostra_classificada['datetime_partida_amostra'])

# Lista de colunas a serem atualizadas
colunas_para_atualizar = ['servico_apurado', 'id_veiculo_apurado', 'sentido_apurado', 
                          'datetime_partida_apurado', 'datetime_chegada_apurado']

for i in range(1, len(amostra_classificada) - 1):  # Excluindo a última linha para evitar problemas de indexação
    ano_atual = amostra_classificada.loc[i, 'datetime_partida_amostra'].year
    sentido_anterior = amostra_classificada.loc[i - 1, 'sentido_amostra']
    status_anterior = amostra_classificada.loc[i - 1, 'status']

    if ano_atual == 2023 and sentido_anterior == 'V' and status_anterior != 'NA':
        amostra_classificada.loc[i, 'status'] = 'Viagem circular identificada e já paga'
        
        # Atualizar as outras colunas com os valores da linha de cima ou de baixo
        for col in colunas_para_atualizar:
            valor_de_cima = amostra_classificada.loc[i - 1, col]
            valor_de_baixo = amostra_classificada.loc[i + 1, col]
            
            if pd.isna(valor_de_cima):
                amostra_classificada.loc[i, col] = valor_de_baixo
            else:
                amostra_classificada.loc[i, col] = valor_de_cima
            

Apenas três viagens de 2023 não foram identificadas até esta etapa e nenhuma delas foi encontrada nas viagens em conformidade:
- 2023-07-06 18:53:00
- 2023-07-11 18:52:00
- 2023-07-19 18:49:00

#### 3.3 Verificar dados de GPS (Tabela `gps_sppo`)

Considerando que foram identificadas apenas 3 viagens em 2022, sugerindo a necessidade de reprocessamento para o ano, aqui serão verificados os dados de GPS apenas de 2023.

In [13]:
amostra_classificada['data'] = pd.to_datetime(amostra_classificada['data'])
amostra_classificada_2023 = amostra_classificada[amostra_classificada['data'].dt.year == 2023]
amostra_classificada_2022 = amostra_classificada[amostra_classificada['data'].dt.year == 2022]

In [14]:
amostra_sem_status_2023 = amostra_classificada_2023[pd.isna(amostra_classificada_2023['status'])]
amostra_sem_status_2023

,data,servico_amostra,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
185,2023-07-06,844,47689,18:53:00,19:06:00,V,2023-07-06 18:53:00,2023-07-06 19:06:00,NaN,NaN,NaN,NaN,NaT,NaT
235,2023-07-11,844,47689,18:52:00,19:00:00,V,2023-07-11 18:52:00,2023-07-11 19:00:00,NaN,NaN,NaN,NaN,NaT,NaT
279,2023-07-19,844,47689,18:49:00,18:58:00,V,2023-07-19 18:49:00,2023-07-19 18:58:00,NaN,NaN,NaN,NaN,NaT,NaT


In [15]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2023-07-19"
  AND servico = '844'
"""
       
shape_identificado = bd.read_sql(q, from_file=True)
shape_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  6.06rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [16]:
q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = '2023-07-19'
  AND servico = '844'
  AND timestamp_gps BETWEEN "2023-07-19T18:49:00"
  AND "2023-07-19T20:06:00"
"""

gps_n_identificado = bd.read_sql(q, from_file=True)
gps_n_identificado.info() 

Downloading: 100%|██████████| 154/154 [00:00<00:00, 384.05rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           154 non-null    object        
 1   servico              154 non-null    object        
 2   timestamp_gps        154 non-null    datetime64[ns]
 3   posicao_veiculo_geo  154 non-null    object        
dtypes: datetime64[ns](1), object(3)
memory usage: 4.9+ KB


In [17]:
map = create_trip_map(gps_n_identificado, shape_identificado)
map.save('./../data/figures/maps/844 19-07-2023 18h53m.html')
map

Após a verificação dos sinais de GPS, a viagem do dia 11 não retornou sinal de GPS no horário indicado.
Nas viagens dos dias 06 e 19, o veículo não passou no raio de 500m da viagem circular.

In [18]:
amostra_classificada.loc[amostra_classificada['datetime_partida_amostra'].isin(['2023-07-06 18:53:00', '2023-07-19 18:49:00']), 'status'] = 'Viagem circular inválida - sem sinal inicial/final dentro do raio de 500m'
amostra_classificada.loc[amostra_classificada['datetime_partida_amostra'] == '2023-07-11 18:52:00', 'status'] = 'Viagem inválida - sinal de GPS não encontrado no momento indicado na amostra.'

In [19]:
amostra_classificada.to_excel('./../data/output/analise_amostra_pre_solucao.xlsx')

In [20]:
# Convert the 'data' column to datetime format
# amostra_classificada['data'] = pd.to_datetime(amostra_classificada['data'])

# Filter for rows where 'data' is in 2022
amostra_2022 = amostra_classificada[amostra_classificada['data'].dt.year == 2022]

# Filter for rows where 'data' is in 2023
amostra_2023 = amostra_classificada[amostra_classificada['data'].dt.year == 2023]


# Count non-NaN values in the 'status' column for each filtered DataFrame
non_nan_count_2022 = amostra_2022['status'].notna().sum()
non_nan_count_2023 = amostra_2023['status'].notna().sum()

print(f"Viagens classificadas em 2022: {non_nan_count_2022}")
print(f"Viagens classificadas em 2022 em %: {round(non_nan_count_2022 / amostra_2022.shape[0],2)*100} %")

print(f"Viagens classificadas em 2023: {non_nan_count_2023}")
print(f"Viagens classificadas em 2023: {round(non_nan_count_2023 / amostra_2023.shape[0],2)*100} %")

Viagens classificadas em 2022: 3
Viagens classificadas em 2022 em %: 2.0 %
Viagens classificadas em 2023: 144
Viagens classificadas em 2023: 100.0 %


Foram identificadas 147 das 280 viagens válidas da amostra. Nestes casos, o veículo que realizou a viagem é o mesmo veículo que consta no amostra. 

## (2) Gráficos da análise exploratória / análise histórica da linha

- Gráfico POD até a data mais recente (quando fazemos a avaliação): `analise_pod_historico.png`

In [21]:
# POD pré-reprocessamento
q = f"""
SELECT
  data, viagens, perc_km_planejada
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico = '{servico}'
  AND DATA BETWEEN '2022-06-01' AND '2023-08-31'
"""
pod_pre_reprocessamento = bd.read_sql(q, from_file=True)

Downloading: 100%|██████████| 381/381 [00:00<00:00, 1060.53rows/s]


In [22]:
# Assegurando que a coluna 'data' seja do tipo datetime
pod_pre_reprocessamento = pod_pre_reprocessamento.sort_values('data')
pod_pre_reprocessamento['data'] = pd.to_datetime(pod_pre_reprocessamento['data'])
# Criando a nova coluna 'mes_ano'
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['data'].dt.to_period('M')
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['mes_ano'].astype(str)
pod_pre_reprocessamento

# Calculando a mediana, Q1 e Q3 para pod_pre_reprocessamento
median_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].median()
q1_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.25)
q3_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].quantile(0.75)

In [23]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionando área sombreada para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=q1_pre.values, 
                         line=dict(width=0), fill=None, mode='lines', name='Q1 Pre'))
fig.add_trace(go.Scatter(x=median_pre.index, y=q3_pre.values, 
                         fill='tonexty', fillcolor='rgba(30,144,255,0.3)', mode='lines', name='Q3 Pre', 
                         line=dict(width=0))) 

# Adicionando a linha da mediana para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=median_pre.index, y=median_pre.values,
                    mode='lines+markers',
                    name='Mediana Antes do Reprocessamento',
                    marker=dict(color='dodgerblue'),
                    opacity=0.75))

# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Percentual de Operação Diária (POD) por mês - Serviço: {}".format(servico), font=dict(color='black')),
    showlegend=False,
    yaxis_range=[0, 150],
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Mediana, 1º e 3º quartis mensais do POD',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        )
    ]
)

# Adicionar linha horizontal para indicar o mínimo de 80%
fig.add_hline(y=80, annotation_text="min = 80%")

fig.show()

# (3) Análise das viagens da amostra x apuradas (pós-solução)

- Tabela de viagens identificadas da amostra: `output/analise_amostra_pos_solucao.xlsx`

- Os dados reprocessados para o ano de 2022 estão disponíveis no dataset: `rj-smtr-dev.SMTR202211008775_reprocessamento`

Solução: reprocessamento das viagens de 2022 


> data	servico	sentido	id_veiculo	datetime_partida_amostra	datetime_chegada_amostra	datetime_partida_apuracao_solucao	datetime_chegada_apuracao_solucao	status

#### 3.1 Checar viagens completas reprocessadas para 2022

In [24]:
q = f"""
SELECT
  data,
  id_veiculo,
  servico_informado,
  sentido,
  datetime_partida,
  datetime_chegada
FROM
  `rj-smtr-dev.SMTR202211008775_reprocessamento.viagem_completa`
WHERE
  DATA IN {tuple(datas)}
"""
viagens_reprocessadas = bd.read_sql(q, from_file=True)
viagens_reprocessadas.info()   

viagens_reprocessadas['id_veiculo'] = viagens_reprocessadas['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)
viagens_reprocessadas['data'] = pd.to_datetime(viagens_reprocessadas['data'])

viagens_reprocessadas['datetime_partida'] = pd.to_datetime(viagens_reprocessadas['datetime_partida'])
viagens_reprocessadas['datetime_chegada'] = pd.to_datetime(viagens_reprocessadas['datetime_chegada'])

Downloading: 100%|██████████| 60/60 [00:00<00:00, 182.37rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   data               60 non-null     dbdate        
 1   id_veiculo         60 non-null     object        
 2   servico_informado  60 non-null     object        
 3   sentido            60 non-null     object        
 4   datetime_partida   60 non-null     datetime64[ns]
 5   datetime_chegada   60 non-null     datetime64[ns]
dtypes: datetime64[ns](2), dbdate(1), object(3)
memory usage: 2.9+ KB


In [25]:
amostra_deduplicada['data'] = pd.to_datetime(amostra_deduplicada['data'])
amostra_2022 = amostra_deduplicada[amostra_deduplicada['data'].dt.year == 2022]
amostra_2023 = amostra_deduplicada[amostra_deduplicada['data'].dt.year == 2023]
amostra_2022['status'] = np.nan
amostra_2022

,data,servico,id_veiculo,hora_inicio,hora_fim,sentido,datetime_partida,datetime_chegada,status
128,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00,NaN
129,2022-09-15,844,47689,10:42:00,10:50:00,V,2022-09-15 10:42:00,2022-09-15 10:50:00,NaN
130,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00,NaN
131,2022-09-15,844,47689,11:03:00,11:10:00,V,2022-09-15 11:03:00,2022-09-15 11:10:00,NaN
132,2022-09-15,844,47689,11:10:00,11:21:00,I,2022-09-15 11:10:00,2022-09-15 11:21:00,NaN
...,...,...,...,...,...,...,...,...,...
259,2022-09-28,844,47689,18:13:00,18:21:00,V,2022-09-28 18:13:00,2022-09-28 18:21:00,NaN
260,2022-09-28,844,47689,18:21:00,18:36:00,I,2022-09-28 18:21:00,2022-09-28 18:36:00,NaN
261,2022-09-28,844,47689,18:36:00,18:45:00,V,2022-09-28 18:36:00,2022-09-28 18:45:00,NaN
262,2022-09-28,844,47689,18:45:00,18:58:00,I,2022-09-28 18:45:00,2022-09-28 18:58:00,NaN


In [26]:
amostra_2022_completa_reprocessada = check_trips(amostra_2022, viagens_reprocessadas, 8, "Viagem circular identificada após o reprocessamento")
amostra_2022_completa_reprocessada

amostra_2022_completa_reprocessada['status'].notna().sum()

# checar se foi repetido mais de 2 vezes
repeated_values = amostra_2022_completa_reprocessada['datetime_partida_apurado'].value_counts()
exists_greater_than_two = any(repeated_values > 2)

if exists_greater_than_two:
    print("Existem valores que aparecem mais de duas vezes.")
else:
    print("Não existem valores que aparecem mais de duas vezes.")


Casos duplicados encontrados no cruzamento de dados:
    data_amostra servico_amostra id_veiculo_amostra hora_inicio  hora_fim  \
0     2022-09-15             844              47689    10:42:00  10:50:00   
1     2022-09-15             844              47689    10:51:00  11:02:00   
2     2022-09-15             844              47689    11:03:00  11:10:00   
3     2022-09-15             844              47689    11:10:00  11:21:00   
4     2022-09-15             844              47689    11:22:00  11:29:00   
..           ...             ...                ...         ...       ...   
113   2022-09-28             844              47689    17:59:00  18:13:00   
114   2022-09-28             844              47689    18:13:00  18:21:00   
115   2022-09-28             844              47689    18:21:00  18:36:00   
116   2022-09-28             844              47689    18:36:00  18:45:00   
117   2022-09-28             844              47689    18:45:00  18:58:00   

    sentido_amostra d

In [27]:
# teve um caso de viagem completa que não foi identificado no passo acima e foi inserido manualmente abaixo
amostra_2022_completa_reprocessada.loc[amostra_2022_completa_reprocessada['datetime_partida_amostra'] == '2022-09-28 16:41:00', 'status'] = 'Viagem circular identificada após o reprocessamento'


#### 3.2 Checar as viagens conformidade reprocessadas para 2022

In [28]:
q = f"""
SELECT
  data,
  id_veiculo,
  servico_informado,
  sentido,
  datetime_partida,
  datetime_chegada
FROM
  `rj-smtr-dev.SMTR202211008775_reprocessamento.viagem_conformidade`
WHERE
  DATA IN {tuple(datas)}
"""
viagens_reprocessadas_conformidade = bd.read_sql(q, from_file=True)
viagens_reprocessadas_conformidade.info()   

viagens_reprocessadas_conformidade['id_veiculo'] = viagens_reprocessadas_conformidade['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)
viagens_reprocessadas_conformidade['data'] = pd.to_datetime(viagens_reprocessadas_conformidade['data'])

Downloading: 100%|██████████| 64/64 [00:00<00:00, 88.64rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   data               64 non-null     dbdate        
 1   id_veiculo         64 non-null     object        
 2   servico_informado  64 non-null     object        
 3   sentido            64 non-null     object        
 4   datetime_partida   64 non-null     datetime64[ns]
 5   datetime_chegada   64 non-null     datetime64[ns]
dtypes: datetime64[ns](2), dbdate(1), object(3)
memory usage: 3.1+ KB


In [29]:

amostra_2022_completa_reprocessada2 = check_trips(amostra_2022_completa_reprocessada, viagens_reprocessadas_conformidade, 6, 
                                                  "Viagem circular inválida - não atendeu o percentual de conformidade do GPS ou do trajeto")


# checar se foi repetido mais de 2 vezes
repeated_values = amostra_2022_completa_reprocessada['datetime_partida_apurado'].value_counts()
exists_greater_than_two = any(repeated_values > 2)

if exists_greater_than_two:
    print("Existem valores que aparecem mais de duas vezes.")
else:
    print("Não existem valores que aparecem mais de duas vezes.")

amostra_2022_completa_reprocessada2


Casos duplicados encontrados no cruzamento de dados:
  data_amostra servico_amostra id_veiculo_amostra hora_inicio  hora_fim  \
0   2022-09-28             844              47689    13:23:00  13:31:00   
1   2022-09-28             844              47689    13:31:00  13:48:00   

  sentido_amostra datetime_partida_amostra datetime_chegada_amostra  \
0               V      2022-09-28 13:23:00      2022-09-28 13:31:00   
1               I      2022-09-28 13:31:00      2022-09-28 13:48:00   

                                              status data_apurado  \
0  Viagem circular inválida - não atendeu o perce...   2022-09-28   
1  Viagem circular inválida - não atendeu o perce...   2022-09-28   

  id_veiculo_apurado servico_apurado sentido_apurado datetime_partida_apurado  \
0              47689             844               C      2022-09-28 13:25:24   
1              47689             844               C      2022-09-28 13:25:24   

  datetime_chegada_apurado  
0      2022-09-28 13:42:5

,data,servico_amostra,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,data_amostra,data_apurado,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
0,2022-09-28,844,47689,16:41:00,16:58:00,I,2022-09-28 16:41:00,2022-09-28 16:58:00,Viagem circular identificada após o reprocessa...,NaT,NaT,NaN,NaN,NaN,NaT,NaT
1,2022-09-15,844,47689,10:42:00,10:50:00,V,2022-09-15 10:42:00,2022-09-15 10:50:00,Viagem circular identificada após o reprocessa...,2022-09-15 00:00:00,2022-09-15,47689,844,C,2022-09-15 10:43:54,2022-09-15 10:57:54
2,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00,Viagem circular identificada após o reprocessa...,2022-09-15 00:00:00,2022-09-15,47689,844,C,2022-09-15 10:43:54,2022-09-15 10:57:54
3,2022-09-15,844,47689,11:03:00,11:10:00,V,2022-09-15 11:03:00,2022-09-15 11:10:00,Viagem circular identificada após o reprocessa...,2022-09-15 00:00:00,2022-09-15,47689,844,C,2022-09-15 11:05:24,2022-09-15 11:16:54
4,2022-09-15,844,47689,11:10:00,11:21:00,I,2022-09-15 11:10:00,2022-09-15 11:21:00,Viagem circular identificada após o reprocessa...,2022-09-15 00:00:00,2022-09-15,47689,844,C,2022-09-15 11:05:24,2022-09-15 11:16:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,2022-09-15,844,47689,11:40:00,11:50:00,V,2022-09-15 11:40:00,2022-09-15 11:50:00,Viagem circular inválida - não atendeu o perce...,2022-09-15,2022-09-15,47689,844,C,2022-09-15 11:41:54,2022-09-15 11:57:54
132,2022-09-20,844,47689,14:04:00,14:20:00,V,2022-09-20 14:04:00,2022-09-20 14:20:00,Viagem circular inválida - não atendeu o perce...,2022-09-20,2022-09-20,47689,844,C,2022-09-20 14:07:24,2022-09-20 14:27:24
133,2022-09-20,844,47689,17:24:00,17:35:00,V,2022-09-20 17:24:00,2022-09-20 17:35:00,Viagem circular inválida - não atendeu o perce...,2022-09-20,2022-09-20,47689,844,C,2022-09-20 17:27:24,2022-09-20 17:49:24
134,2022-09-28,844,47689,13:23:00,13:31:00,V,2022-09-28 13:23:00,2022-09-28 13:31:00,Viagem circular inválida - não atendeu o perce...,2022-09-28,2022-09-28,47689,844,C,2022-09-28 13:25:24,2022-09-28 13:42:54


In [30]:
amostra_classificada_2023 = amostra_classificada[amostra_classificada['data'].dt.year == 2023]
amostra_reprocessada = pd.concat([amostra_2022_completa_reprocessada2, amostra_classificada_2023], ignore_index=True)
amostra_reprocessada

,data,servico_amostra,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,data_amostra,data_apurado,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
0,2022-09-28,844,47689,16:41:00,16:58:00,I,2022-09-28 16:41:00,2022-09-28 16:58:00,Viagem circular identificada após o reprocessa...,NaT,NaT,NaN,NaN,NaN,NaT,NaT
1,2022-09-15,844,47689,10:42:00,10:50:00,V,2022-09-15 10:42:00,2022-09-15 10:50:00,Viagem circular identificada após o reprocessa...,2022-09-15 00:00:00,2022-09-15,47689,844,C,2022-09-15 10:43:54,2022-09-15 10:57:54
2,2022-09-15,844,47689,10:51:00,11:02:00,I,2022-09-15 10:51:00,2022-09-15 11:02:00,Viagem circular identificada após o reprocessa...,2022-09-15 00:00:00,2022-09-15,47689,844,C,2022-09-15 10:43:54,2022-09-15 10:57:54
3,2022-09-15,844,47689,11:03:00,11:10:00,V,2022-09-15 11:03:00,2022-09-15 11:10:00,Viagem circular identificada após o reprocessa...,2022-09-15 00:00:00,2022-09-15,47689,844,C,2022-09-15 11:05:24,2022-09-15 11:16:54
4,2022-09-15,844,47689,11:10:00,11:21:00,I,2022-09-15 11:10:00,2022-09-15 11:21:00,Viagem circular identificada após o reprocessa...,2022-09-15 00:00:00,2022-09-15,47689,844,C,2022-09-15 11:05:24,2022-09-15 11:16:54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,2023-07-19 00:00:00,844,47689,17:48:00,17:59:00,V,2023-07-19 17:48:00,2023-07-19 17:59:00,Viagem circular identificada e já paga,NaN,NaT,47689,844,C,2023-07-19 17:48:39,2023-07-19 18:11:09
276,2023-07-19 00:00:00,844,47689,18:01:00,18:16:00,I,2023-07-19 18:01:00,2023-07-19 18:16:00,Viagem circular identificada e já paga,NaN,NaT,47689,844,C,2023-07-19 17:48:39,2023-07-19 18:11:09
277,2023-07-19 00:00:00,844,47689,18:18:00,18:33:00,V,2023-07-19 18:18:00,2023-07-19 18:33:00,Viagem circular identificada e já paga,NaN,NaT,47689,844,C,2023-07-19 18:21:09,2023-07-19 18:42:39
278,2023-07-19 00:00:00,844,47689,18:35:00,18:48:00,I,2023-07-19 18:35:00,2023-07-19 18:48:00,Viagem circular identificada e já paga,NaN,NaT,47689,844,C,2023-07-19 18:21:09,2023-07-19 18:42:39


#### 3.3 Checar dados de GPS

In [31]:
amostra_sem_viagem = amostra_reprocessada[amostra_reprocessada['status'].isna()]
amostra_sem_viagem

data_n_encontradas_gps = amostra_sem_viagem['data'].unique()
data_n_encontradas_gps = pd.to_datetime(pd.Series(data_n_encontradas_gps))
data_n_encontradas_gps = data_n_encontradas_gps.dt.strftime('%Y-%m-%d')
data_n_encontradas_gps = data_n_encontradas_gps.values
data_n_encontradas_gps

q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA IN {tuple(data_n_encontradas_gps)}
  AND id_veiculo = "C47689"
"""
          
dados_gps = bd.read_sql(q, from_file=True)
dados_gps.info()
dados_gps['id_veiculo'] = dados_gps['id_veiculo'].astype(str).apply(lambda x: x[1:] if x[0].isalpha() else x)

Downloading: 100%|██████████| 8640/8640 [00:00<00:00, 9632.05rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8640 entries, 0 to 8639
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           8640 non-null   object        
 1   servico              8640 non-null   object        
 2   timestamp_gps        8640 non-null   datetime64[ns]
 3   posicao_veiculo_geo  8640 non-null   object        
dtypes: datetime64[ns](1), object(3)
memory usage: 270.1+ KB


In [32]:
amostra_sem_viagem

,data,servico_amostra,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,data_amostra,data_apurado,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
120,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT
121,2022-09-15,844,47689,11:51:00,12:02:00,I,2022-09-15 11:51:00,2022-09-15 12:02:00,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT
122,2022-09-15,844,47689,18:59:00,19:12:00,V,2022-09-15 18:59:00,2022-09-15 19:12:00,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT
123,2022-09-20,844,47689,10:26:00,10:37:00,I,2022-09-20 10:26:00,2022-09-20 10:37:00,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT
124,2022-09-20,844,47689,14:20:00,14:34:00,I,2022-09-20 14:20:00,2022-09-20 14:34:00,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT
125,2022-09-20,844,47689,17:35:00,17:57:00,I,2022-09-20 17:35:00,2022-09-20 17:57:00,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT
126,2022-09-20,844,47689,19:03:00,19:16:00,V,2022-09-20 19:03:00,2022-09-20 19:16:00,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT
127,2022-09-28,844,47689,10:46:00,10:52:00,I,2022-09-28 10:46:00,2022-09-28 10:52:00,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT
128,2022-09-28,844,47689,12:11:00,12:22:00,V,2022-09-28 12:11:00,2022-09-28 12:22:00,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT
129,2022-09-28,844,47689,12:22:00,12:34:00,I,2022-09-28 12:22:00,2022-09-28 12:34:00,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaT


In [33]:
def set_status(row, df_check):
    # Filter the df_check by vehicle ID and time range
    filtered_df = df_check[
        (df_check['id_veiculo'] == row['id_veiculo_amostra']) & 
        (df_check['timestamp_gps'] >= row['datetime_partida_amostra']) & 
        (df_check['timestamp_gps'] <= row['datetime_chegada_amostra'])
    ]
    
    # If there are rows in the filtered dataframe, check the condition
    if not filtered_df.empty:
        if all(filtered_df['servico'] == row['servico_amostra']) and np.isnan(row['status']):            
            return "Viagem circular inválida - sem sinal inicial/final dentro do raio de 500m"
        else:
            return "Sinal de GPS não encontrado para o veículo no horário da viagem"
            
    return row['status']

amostra_sem_viagem['status'] = amostra_sem_viagem.apply(lambda row: set_status(row, dados_gps), axis=1)

amostra_sem_viagem

,data,servico_amostra,id_veiculo_amostra,hora_inicio,hora_fim,sentido_amostra,datetime_partida_amostra,datetime_chegada_amostra,status,data_amostra,data_apurado,id_veiculo_apurado,servico_apurado,sentido_apurado,datetime_partida_apurado,datetime_chegada_apurado
120,2022-09-15,844,47689,10:32:00,10:40:00,I,2022-09-15 10:32:00,2022-09-15 10:40:00,Viagem circular inválida - sem sinal inicial/f...,NaN,NaT,NaN,NaN,NaN,NaT,NaT
121,2022-09-15,844,47689,11:51:00,12:02:00,I,2022-09-15 11:51:00,2022-09-15 12:02:00,Viagem circular inválida - sem sinal inicial/f...,NaN,NaT,NaN,NaN,NaN,NaT,NaT
122,2022-09-15,844,47689,18:59:00,19:12:00,V,2022-09-15 18:59:00,2022-09-15 19:12:00,Viagem circular inválida - sem sinal inicial/f...,NaN,NaT,NaN,NaN,NaN,NaT,NaT
123,2022-09-20,844,47689,10:26:00,10:37:00,I,2022-09-20 10:26:00,2022-09-20 10:37:00,Viagem circular inválida - sem sinal inicial/f...,NaN,NaT,NaN,NaN,NaN,NaT,NaT
124,2022-09-20,844,47689,14:20:00,14:34:00,I,2022-09-20 14:20:00,2022-09-20 14:34:00,Viagem circular inválida - sem sinal inicial/f...,NaN,NaT,NaN,NaN,NaN,NaT,NaT
125,2022-09-20,844,47689,17:35:00,17:57:00,I,2022-09-20 17:35:00,2022-09-20 17:57:00,Viagem circular inválida - sem sinal inicial/f...,NaN,NaT,NaN,NaN,NaN,NaT,NaT
126,2022-09-20,844,47689,19:03:00,19:16:00,V,2022-09-20 19:03:00,2022-09-20 19:16:00,Viagem circular inválida - sem sinal inicial/f...,NaN,NaT,NaN,NaN,NaN,NaT,NaT
127,2022-09-28,844,47689,10:46:00,10:52:00,I,2022-09-28 10:46:00,2022-09-28 10:52:00,Viagem circular inválida - sem sinal inicial/f...,NaN,NaT,NaN,NaN,NaN,NaT,NaT
128,2022-09-28,844,47689,12:11:00,12:22:00,V,2022-09-28 12:11:00,2022-09-28 12:22:00,Viagem circular inválida - sem sinal inicial/f...,NaN,NaT,NaN,NaN,NaN,NaT,NaT
129,2022-09-28,844,47689,12:22:00,12:34:00,I,2022-09-28 12:22:00,2022-09-28 12:34:00,Viagem circular inválida - sem sinal inicial/f...,NaN,NaT,NaN,NaN,NaN,NaT,NaT


Em todos os casos acima, foram encontrados sinais de GPS para os veículos, o que indica que eles tiveram um problema relacionado ao trajeto das viagens.

In [34]:
amostra_com_viagem = amostra_reprocessada[~amostra_reprocessada['status'].isna()]
# juntar as duas 
result = pd.concat([amostra_com_viagem, amostra_sem_viagem], ignore_index=True)
result.to_excel('.././data/output/analise_amostra_pos_solucao.xlsx')

In [35]:
result['data'] = pd.to_datetime(result['data'], errors='coerce')
amostra_reprocess_2022 = result[result['data'].dt.year == 2022]
non_nan_count_2022 = amostra_reprocess_2022['status'].notna().sum()
non_nan_count_2022
print(f"Viagens classificadas em 2022: {non_nan_count_2022}")
print(f"Viagens classificadas em 2022 em %: {round(non_nan_count_2022 / amostra_2022.shape[0],2)*100} %")

Viagens classificadas em 2022: 136
Viagens classificadas em 2022 em %: 100.0 %


### Exemplos (mapa)

- Mapa de 1 viagem identificada (que não foi identificada antes da
  solução): `analise_mapa_viagem_identificada_pos_solucao.html`
- Mapa de viagens não identificadas (se necessário i.e. teve sinal de
  GPS na linha no período, mas não teve a viagem): `analise_mapa_viagem_nao_identificada_pos_solucao_[descricao].html`

Exemplo de viagem não encontrada em que apenas metade do trajeto foi feito:

In [36]:
q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = '2022-09-28'
  AND id_veiculo = "C47689"
  AND timestamp_gps BETWEEN "2022-09-28T10:46:00" AND "2022-09-28T10:52:00"
"""     
dados_gps = bd.read_sql(q, from_file=True)
dados_gps.info()

Downloading: 100%|██████████| 12/12 [00:00<00:00, 23.58rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           12 non-null     object        
 1   servico              12 non-null     object        
 2   timestamp_gps        12 non-null     datetime64[ns]
 3   posicao_veiculo_geo  12 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 512.0+ bytes


In [37]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-09-28"
  AND servico = '844'
"""
       
shape_n_identificado = bd.read_sql(q, from_file=True)
shape_n_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  5.67rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [38]:
create_trip_map(dados_gps, shape_identificado)

Exemplo de viagem não encontrada em que 

47689	12:11:00	12:22:00	V	2022-09-28 12:11:00	2022-09-28 12:22:00	Encontrado sinal de GPS do veículo no horário ...	NaN	NaT	NaN	NaN	NaN	NaT	NaT
129	2022-09-28	844	47689	12:22:00	12:34:00	I	2022-09-28 12:22:00	2022-09-28 12:34:00

In [39]:
q = f"""
  SELECT
  id_veiculo,
  servico,
  timestamp_gps,
  ST_GEOGPOINT(longitude, latitude) as posicao_veiculo_geo
FROM
  `rj-smtr.br_rj_riodejaneiro_veiculos.gps_sppo`
WHERE
  DATA = '2022-09-28'
  AND id_veiculo = "C47689"
  AND timestamp_gps BETWEEN "2022-09-28T12:11:00" AND "2022-09-28T12:34:00"
"""     
dados_gps = bd.read_sql(q, from_file=True)
dados_gps.info()

Downloading: 100%|██████████| 46/46 [00:00<00:00, 139.39rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id_veiculo           46 non-null     object        
 1   servico              46 non-null     object        
 2   timestamp_gps        46 non-null     datetime64[ns]
 3   posicao_veiculo_geo  46 non-null     object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1.6+ KB


In [40]:
q = f"""
SELECT
  shape_id,
  shape,
  start_pt,
  end_pt
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
WHERE
  DATA = "2022-09-28"
  AND servico = '844'
"""
       
shape_n_identificado = bd.read_sql(q, from_file=True)
shape_n_identificado.info() 

Downloading: 100%|██████████| 2/2 [00:00<00:00,  6.39rows/s]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   shape_id  2 non-null      object
 1   shape     2 non-null      object
 2   start_pt  2 non-null      object
 3   end_pt    2 non-null      object
dtypes: object(4)
memory usage: 192.0+ bytes


In [41]:
create_trip_map(dados_gps, shape_identificado)